In [1]:
import torch
import torch.optim as optim
import pandas as pd
import numpy as np

In [2]:
%matplotlib inline


In [3]:
RETOS_BEBRASK_dataset = pd.read_excel("RETOS_BEBRASK_long.xlsx")

In [4]:
rating_columns = [col for col in RETOS_BEBRASK_dataset.columns if 'Rating0' in col]
fulfilled_columns = [col for col in RETOS_BEBRASK_dataset.columns if 'Fulfilled' in col]
actions_dataset = RETOS_BEBRASK_dataset[rating_columns]-1
states_dataset = RETOS_BEBRASK_dataset[fulfilled_columns]

In [5]:


# Assuming data for each subject is grouped together
batches_per_subject = 5

# Dictionary to hold training and testing data for cross-validation
cross_val_data = {}

# Use enumerate to get index and subject data from actions and states datasets
for i, (actions, states) in enumerate(zip(actions_dataset.values, states_dataset.values)):
    num_actions = len(actions)
    batch_size = num_actions // batches_per_subject
    cross_val_data[i] = []

    for j in range(batches_per_subject):
        start_index = j * batch_size
        if j == batches_per_subject - 1:
            end_index = num_actions  # Ensure the last batch goes up to the end
        else:
            end_index = start_index + batch_size
        
        # Test data for the current fold
        test_data = (states[start_index:end_index], actions[start_index:end_index])

        # Training data for the current fold
        # Combine slices before and after the test segment
        train_states = np.concatenate((states[:start_index], states[end_index:]))
        train_actions = np.concatenate((actions[:start_index], actions[end_index:]))
        train_data = (train_states, train_actions)

        # Save the train and test data in the dictionary
        cross_val_data[i].append((train_data, test_data))

# Now, `cross_val_data` is ready for use in your training/testing loops


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

class BCModel(nn.Module):
    def __init__(self):
        super(BCModel, self).__init__()
        self.fc1 = nn.Linear(2, 32)
        self.dropout1 = nn.Dropout(0.5)  # Dropout with a probability of 0.5
        self.fc2 = nn.Linear(32, 32)
        self.dropout2 = nn.Dropout(0.5)  # Dropout with a probability of 0.5
        self.fc3 = nn.Linear(32, 4)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return torch.softmax(x, dim=1)


In [7]:
model = BCModel()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

all_subjects_avg_deviation = []
all_subjects_last_epoch_loss = []
all_subjects_test_deviation = []
all_subjects_custom_loss = []


train_len = 36
test_len = 9

for subject, data_splits in cross_val_data.items():
    print(f"Training for Subject {subject}")
    subject_outputs = []
    subject_avg_deviation = []
    last_epoch_losses = []
    test_deviation = []
    custom_losses = []
    for fold, ((train_states, train_actions), (test_states, test_actions)) in enumerate(data_splits):
        # Data preparation includes the first state treatment
        train_states_expanded = np.column_stack((train_states, np.roll(train_states, 1)))
        train_states_expanded[0, 1] = train_states_expanded[0, 0]
        
        test_states_expanded = np.column_stack((test_states, np.roll(test_states, 1)))
        test_states_expanded[0, 1] = test_states_expanded[0, 0]

        # Convert to tensors and create dataloaders
        train_states_tensor = torch.tensor(train_states_expanded, dtype=torch.float32)
        train_actions_tensor = torch.tensor(train_actions, dtype=torch.long)
        test_states_tensor = torch.tensor(test_states_expanded, dtype=torch.float32)
        test_actions_tensor = torch.tensor(test_actions, dtype=torch.long)

        train_dataset = TensorDataset(train_states_tensor, train_actions_tensor)
        train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
        test_dataset = TensorDataset(test_states_tensor, test_actions_tensor)
        test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

        # Training loop
        model.train()
        
        for epoch in range(50):
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = -torch.sum(torch.log(outputs.gather(1, labels.unsqueeze(1)).squeeze(1)))
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            if epoch == 49:  # Store the last epoch loss
                last_epoch_losses.append(running_loss / train_len)
        
        # Evaluate on test set and compute custom loss
        model.eval()
        test_total_deviation = 0.0
        test_num_samples = 0
        fold_custom_loss = 0.0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted_classes = torch.max(outputs, 1)
                deviation = torch.abs(predicted_classes.float() - labels.float())
                test_total_deviation += deviation.sum().item()
                test_num_samples += labels.size(0)
                
                # Compute the custom loss for selected probabilities
                selected_probabilities = outputs.gather(1, labels.unsqueeze(1)).squeeze(1)
                custom_loss = -torch.sum(torch.log(selected_probabilities))
                fold_custom_loss += custom_loss.item()

        fold_test_deviation = test_total_deviation / test_num_samples
        test_deviation.append(fold_test_deviation)
        custom_losses.append(fold_custom_loss / test_num_samples)

        # Evaluate on the whole dataset after training
        with torch.no_grad():
            full_dataset_outputs = model(train_states_tensor)
            _, full_dataset_predicted = torch.max(full_dataset_outputs, 1)
            deviation = torch.abs(full_dataset_predicted.float() - train_actions_tensor.float())
            total_deviation = deviation.sum().item()
            num_samples = train_actions_tensor.size(0)
            avg_deviation = total_deviation / num_samples
            subject_avg_deviation.append(avg_deviation)
            subject_outputs.append(full_dataset_outputs.numpy())  # Store outputs for other uses
    
    # After all folds for a subject
    average_of_deviation = np.mean(subject_avg_deviation)
    average_test_deviation = np.mean(test_deviation)
    all_subjects_avg_deviation.append(average_of_deviation)
    all_subjects_test_deviation.append(average_test_deviation)
    average_last_epoch_loss = np.mean(last_epoch_losses)
    all_subjects_last_epoch_loss.append(average_last_epoch_loss)
    average_custom_loss = np.mean(custom_losses)
    all_subjects_custom_loss.append(average_custom_loss)
    
    print(f"Subject {subject} - Train Custom Loss: {np.mean(last_epoch_losses)}")
    print(f"Subject {subject} - Train Dataset Average Deviation: {average_of_deviation}")
    print(f"Subject {subject} - Test Custom Loss: {average_custom_loss}")
    print(f"Subject {subject} - Test Set Average Deviation: {average_test_deviation}")

# After all subjects
print(f"All Subjects - Overall Average Deviation: {np.mean(all_subjects_avg_deviation)}")
print(f"All Subjects - Overall Test Set Average Deviation: {np.mean(all_subjects_test_deviation)}")
print(f"All Subjects - Overall Average Last Epoch Loss: {np.mean(all_subjects_last_epoch_loss)}")
print(f"All Subjects - Overall Custom Loss: {np.mean(all_subjects_custom_loss)}")

Training for Subject 0
Subject 0 - Train Custom Loss: 0.9808379570643109
Subject 0 - Train Dataset Average Deviation: 0.6222222222222222
Subject 0 - Test Custom Loss: 0.9831166055467394
Subject 0 - Test Set Average Deviation: 0.6222222222222222
Training for Subject 1
Subject 1 - Train Custom Loss: 1.1951705879635282
Subject 1 - Train Dataset Average Deviation: 0.6666666666666666
Subject 1 - Test Custom Loss: 1.2377259148491753
Subject 1 - Test Set Average Deviation: 0.6888888888888889
Training for Subject 2


KeyboardInterrupt: 

In [7]:
model = BCModel()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

num_samples = 45

all_subjects_outputs = {}

for i, (states, actions) in enumerate(zip(states_dataset.values, actions_dataset.values)):
    states_expanded = np.column_stack((states, np.roll(states, 1)))
    states_expanded[0, 1] = states_expanded[0, 0]  # First state's previous state set to itself
    
    # Convert to tensors
    states_tensor = torch.tensor(states_expanded, dtype=torch.float32)
    actions_tensor = torch.tensor(actions, dtype=torch.long)
    
    # Create DataLoader
    dataset = TensorDataset(states_tensor, actions_tensor)
    loader = DataLoader(dataset, batch_size=10, shuffle=True)
    
    # Training loop
    model.train()
    for epoch in range(40):
        running_loss = 0.0
        for inputs, labels in loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = -torch.sum(torch.log(outputs.gather(1, labels.unsqueeze(1)).squeeze(1)))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # Evaluation loop for custom inputs
    model.eval()
    custom_inputs = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
    with torch.no_grad():
        custom_outputs = model(custom_inputs)
        all_subjects_outputs[i] = custom_outputs.numpy()

    # Regular evaluation loop
    total_deviation = 0.0
    evaluation_loss = 0.0
    num_samples = len(states)
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            _, predicted_classes = torch.max(outputs, 1)
            deviation = torch.abs(predicted_classes.float() - labels.float())
            total_deviation += deviation.sum().item()
            eval_loss = -torch.sum(torch.log(outputs.gather(1, labels.unsqueeze(1)).squeeze(1)))
            evaluation_loss += eval_loss.item()

    average_deviation = total_deviation / num_samples
    average_evaluation_loss = evaluation_loss / num_samples
    print(f"Subject {i} - Average Deviation: {average_deviation}")
    print(f"Subject {i} - Evaluation Loss: {average_evaluation_loss}")

# Print model outputs for specific inputs after all subjects are processed
print("Model outputs for specific inputs across all subjects:")
for subject, outputs in all_subjects_outputs.items():
    print(f"Subject {subject} Outputs:")
    print(outputs)

Subject 0 - Average Deviation: 0.7333333333333333
Subject 0 - Evaluation Loss: 1.0209438747829862
Subject 1 - Average Deviation: 0.6888888888888889
Subject 1 - Evaluation Loss: 1.2068284776475695
Subject 2 - Average Deviation: 0.5777777777777777
Subject 2 - Evaluation Loss: 1.0195906533135308
Subject 3 - Average Deviation: 0.7777777777777778
Subject 3 - Evaluation Loss: 1.0636436992221407
Subject 4 - Average Deviation: 0.6666666666666666
Subject 4 - Evaluation Loss: 1.1014594184027777
Subject 5 - Average Deviation: 0.5111111111111111
Subject 5 - Evaluation Loss: 1.014130327436659
Subject 6 - Average Deviation: 0.7777777777777778
Subject 6 - Evaluation Loss: 1.0318053669399685
Subject 7 - Average Deviation: 0.6888888888888889
Subject 7 - Evaluation Loss: 1.1226889928181967
Subject 8 - Average Deviation: 1.0222222222222221
Subject 8 - Evaluation Loss: 1.136333253648546
Subject 9 - Average Deviation: 0.7555555555555555
Subject 9 - Evaluation Loss: 0.8984461890326606
Subject 10 - Average D

In [9]:
df_data = {key: [sublist for sublist in value] for key, value in all_subjects_outputs.items()}
df_data

{0: [array([0.28795505, 0.16379717, 0.20148192, 0.3467658 ], dtype=float32),
  array([0.47700673, 0.11373572, 0.17789644, 0.23136109], dtype=float32),
  array([0.11525543, 0.14385   , 0.1609354 , 0.57995915], dtype=float32),
  array([0.12320498, 0.11575945, 0.16230999, 0.5987256 ], dtype=float32)],
 1: [array([0.31164536, 0.32230547, 0.21617943, 0.14986981], dtype=float32),
  array([0.36244816, 0.3256212 , 0.19478193, 0.11714873], dtype=float32),
  array([0.07984774, 0.24552882, 0.37149543, 0.303128  ], dtype=float32),
  array([0.09699935, 0.26569858, 0.36706525, 0.27023688], dtype=float32)],
 2: [array([0.67985535, 0.176637  , 0.09162865, 0.05187899], dtype=float32),
  array([0.7725288 , 0.13232285, 0.0689183 , 0.02622996], dtype=float32),
  array([0.16021807, 0.20068312, 0.31099933, 0.32809952], dtype=float32),
  array([0.10243398, 0.17131346, 0.37347448, 0.35277808], dtype=float32)],
 3: [array([0.6007747 , 0.20085366, 0.05791166, 0.14046003], dtype=float32),
  array([0.70639646, 0.

In [10]:
df_data = {key: [sublist for sublist in value] for key, value in all_subjects_outputs.items()}
df = pd.DataFrame.from_dict(df_data, orient='index')
df.to_excel("outputs_bc.xlsx")


In [15]:
flattened_data = {key: sublist.flatten() for key, sublist in all_subjects_outputs.items()}
df = pd.DataFrame.from_dict(flattened_data, orient='index')


In [17]:
df.to_excel("outputs_bc.xlsx")

In [9]:
train_len = 36
test_len = 9
subject_losses = []
for subject, data_splits in cross_val_data.items():
    if subject == 10:
        break
    print(f"Training for Subject {subject}")
    epoch_train_losses = []
    epoch_test_losses = []
    
    for fold, ((train_states, train_actions), (test_states, test_actions)) in enumerate(data_splits):
        model = BCModel()
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
        
        epoch_train_losses_fold = []
        epoch_test_losses_fold = []
        
        train_states_expanded = np.column_stack((train_states, np.roll(train_states, 1)))
        train_states_expanded[0, 1] = train_states_expanded[0, 0]
        
        test_states_expanded = np.column_stack((test_states, np.roll(test_states, 1)))
        test_states_expanded[0, 1] = test_states_expanded[0, 0]

        train_states_tensor = torch.tensor(train_states_expanded, dtype=torch.float32)
        train_actions_tensor = torch.tensor(train_actions, dtype=torch.long)
        test_states_tensor = torch.tensor(test_states_expanded, dtype=torch.float32)
        test_actions_tensor = torch.tensor(test_actions, dtype=torch.long)

        train_dataset = TensorDataset(train_states_tensor, train_actions_tensor)
        train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
        test_dataset = TensorDataset(test_states_tensor, test_actions_tensor)
        test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

        model.train()
        for epoch in range(75):
            running_train_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = -torch.sum(torch.log(outputs.gather(1, labels.unsqueeze(1)).squeeze(1)))
                loss.backward()
                optimizer.step()
                running_train_loss += loss.item()
            epoch_train_losses_fold.append(running_train_loss / train_len)

            # Evaluate on test set for each epoch
            model.eval()
            running_test_loss = 0.0
            with torch.no_grad():
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    test_loss = -torch.sum(torch.log(outputs.gather(1, labels.unsqueeze(1)).squeeze(1)))
                    running_test_loss += test_loss.item()
            epoch_test_losses_fold.append(running_test_loss / test_len)
        epoch_train_losses.append(epoch_train_losses_fold)
        epoch_test_losses.append(epoch_test_losses_fold)
    # Plotting
    subject_losses.append([epoch_train_losses,epoch_test_losses])
    
    

Training for Subject 0
Training for Subject 1
Training for Subject 2
Training for Subject 3
Training for Subject 4
Training for Subject 5
Training for Subject 6
Training for Subject 7
Training for Subject 8
Training for Subject 9


In [13]:
train_losses = np.mean(np.array(subject_losses[0][0]), axis=0).tolist()
test_losses = np.mean(np.array(subject_losses[0][1]), axis=0).tolist()


In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming subject_losses is a list of lists where each sublist contains train and test losses for a subject
# For example, subject_losses[0][0] contains train losses for subject 0, subject_losses[0][1] contains test losses for subject 0

num_subjects = 10

for i in range(num_subjects):
    print(f'Subject {i}')
    train_losses = np.mean(np.array(subject_losses[i][0]), axis=0)
    test_losses = np.mean(np.array(subject_losses[i][1]), axis=0)
    
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss', color='blue')
    plt.plot(test_losses, label='Test Loss', color='red')
    
    plt.title(f'Subject {i} Train and Test Losses')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(title='Legend')
    plt.grid(True)
    
    # Set the background color to white
    plt.gca().set_facecolor('white')
    plt.gcf().set_facecolor('white')
    
    # Set axis and text colors to black
    plt.gca().tick_params(axis='x', colors='black')
    plt.gca().tick_params(axis='y', colors='black')
    plt.gca().xaxis.label.set_color('black')
    plt.gca().yaxis.label.set_color('black')
    plt.gca().title.set_color('black')
    plt.gca().legend().get_title().set_color('black')
    plt.gca().legend().get_texts()[0].set_color('black')
    plt.gca().legend().get_texts()[1].set_color('black')
    
    plt.show()


In [1]:
train_losses

NameError: name 'train_losses' is not defined